In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
x_train = train
x_test = test

In [4]:
x_train['Date'] = pd.to_datetime(x_train['Date'])
x_train['Date'] = x_train['Date'].dt.strftime("%m%d")
x_test['Date'] = pd.to_datetime(x_test['Date'])
x_test['Date'] = x_test['Date'].dt.strftime("%m%d")

In [5]:
x_train = x_train.fillna('NA')
x_test = x_test.fillna('NA')

['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kaz

In [6]:
country_list = x_train['Country_Region'].unique()


In [49]:
from warnings import filterwarnings

filterwarnings('ignore')

from sklearn import preprocessing

from xgboost import XGBRegressor

encoder = preprocessing.LabelEncoder()

sub = []
for country in country_list:
    province_list = x_train.loc[x_train['Country_Region'] == country].Province_State.unique()
    for province in province_list:
        X_train = x_train.loc[(x_train['Country_Region'] == country) & (x_train['Province_State'] == province),['Date']].astype('int')
        Y_train_c = x_train.loc[(x_train['Country_Region'] == country) & (x_train['Province_State'] == province),['ConfirmedCases']]
        Y_train_f = x_train.loc[(x_train['Country_Region'] == country) & (x_train['Province_State'] == province),['Fatalities']]
        X_test = x_test.loc[(x_test['Country_Region'] == country) & (x_test['Province_State'] == province), ['Date']].astype('int')
        X_forecastId = x_test.loc[(x_test['Country_Region'] == country) & (x_test['Province_State'] == province), ['ForecastId']]
        X_forecastId = X_forecastId.values.tolist()
        X_forecastId = [v[0] for v in X_forecastId]
        model_c = XGBRegressor(n_estimators=1000)
        model_c.fit(X_train, Y_train_c)
        Y_pred_c = model_c.predict(X_test)
        model_f = XGBRegressor(n_estimators=1000)
        model_f.fit(X_train, Y_train_f)
        Y_pred_f = model_f.predict(X_test)
        for j in range(len(Y_pred_c)):
            dic = { 'ForecastId': X_forecastId[j], 'ConfirmedCases': Y_pred_c[j], 'Fatalities': Y_pred_f[j]}
            sub.append(dic)

['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['Australian Capital Territory' 'New South Wales' 'Northern Territory'
 'Queensland' 'South Australia' 'Tasmania' 'Victoria' 'Western Australia']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['Alberta' 'British Columbia' 'Manitoba' 'New Brunswick'
 'Newfoundland and Labrador' 'Northwest Territories' 'Nova Scotia'
 'Ontario' 'Prince Edward Island' 'Quebec' 'Saskatchewan' 'Yukon']
['NA']
['NA']
['NA']
['Anhui' 'Beijing' 'Chongqing' 'Fujian' 'Gansu' 'Guangdong' 'Guangxi'
 'Guizhou' 'Hainan' 'Hebei' 'Heilongjiang' 'Henan' 'Hong Kong' 'Hubei'
 'Hunan' 'Inner Mongolia' 'Jiangsu' 'Jiangxi' 'Jilin' 'Liaoning' 'Macau'
 'Ningxia' 'Qinghai' 'Shaanxi' 'Shandong' 'Shanghai' 'Shanxi' 'Sichuan'
 'Tianjin' 'Tibet' 'Xinjiang' 'Yunnan' 'Zhejiang']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['NA']
['Faroe Islands' 'Greenla

In [50]:
submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False)